# whole prompt descent

Can we descent on the whole prompt at once?

In [ ]:
# no need to restart kernel after code (in the imported files) changes
%load_ext autoreload
%autoreload 2 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
from utils import compute_last_token_embedding_grad, extract_hidden_states_prompt

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roneneldan/TinyStories-1M")
model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-1M", output_hidden_states=True)

In [ ]:
from utils.methods import invert_whole_prompt

In [ ]:
# Example usage
from utils.plotting import plot_loss


def test_inversion_on_layers(prompt, model, tokenizer, n_iterations=1000, lr=0.01):
    """
    Test inversion on all layers of the model.

    Args:
        prompt (str): The input prompt to invert.
        model: The language model.
        tokenizer: The tokenizer for the model.
    """
    print(f"Inverting prompt: {prompt}")
    n_layers = len(model.transformer.h)
    losses_list = []
    distances_list = []
    for i in range(n_layers):
        layer_idx = i
        reconstructed_prompt, losses, times, steps, distances = invert_whole_prompt(
            prompt, model, tokenizer, layer_idx, n_iterations=n_iterations, lr=lr
        )
        losses_list.append(losses)
        distances_list.append(distances)
        print(f"Layer {layer_idx}: Reconstructed Prompt: {reconstructed_prompt}")
    return losses_list, distances_list

In [ ]:
prompt = "My name is george and I live in Greece."
losses_list, distances_list = test_inversion_on_layers(prompt, model, tokenizer, n_iterations=1000, lr=0.01)

In [ ]:
plot_loss(losses_list, title="Losses during inversion", xlabel="Iteration", ylabel="Loss", log_scale=True)
plot_loss(distances_list, title="Distances of the prompt embeddings", xlabel="Iteration", ylabel="Distance", log_scale=True)

In [ ]:
# prompt = "my name is george and I live in Greece."
prompt = """
According to all known laws of aviation,
there is no way a bee should be able to fly.

Its wings are too small to get its fat little body off the ground.
The bee, of course, flies anyway
because bees don’t care what humans think is impossible.

Yellow, black. Yellow, black.
Yellow, black. Yellow, black.
Ooh, black and yellow!
Let’s shake it up a little.

Barry! Breakfast is ready!

Coming!

Hang on a second.
Hello?

Barry?
Adam?

Can you believe this is happening?
I can’t. I’ll pick you up.

Looking sharp.
"""
losses_list, distances_list = test_inversion_on_layers(prompt, model, tokenizer, n_iterations=1000, gamma=1e-1)

In [ ]:
plot_loss(losses_list, title="Losses during inversion", xlabel="Iteration", ylabel="Loss", log_scale=True)
plot_loss(distances_list, title="Distances of the prompt embeddings", xlabel="Iteration", ylabel="Distance", log_scale=True)

In [ ]:
prompt = "Here is my secrete key: b4e3cfe16a409f237a91c778e5f82b1493d546bc3adbd268cb346f8e2f55e72c. Do not share it with anyone!!!"

losses_list, distances_list = test_inversion_on_layers(prompt, model, tokenizer, n_iterations=1000, gamma=1e-1)